In [1]:
#Import libraries
import pyam
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [124]:
def get_data():
    #get IIASA identification
    #workplace IIASA
    iiasa_creds = r"C:\Users\scheifinger\Documents\GitHub\Feasibility_Tool\iiasa_credentials.yml"
    #workplace home
    #iiasa_creds = r"C:\Users\schei\OneDrive\Dokumente\GitHub\Feasibility_Tool\iiasa_credentials.yml"
    pyam.iiasa.Connection(creds=iiasa_creds)

    connections = list(pyam.iiasa.Connection(creds=iiasa_creds).valid_connections)
    #other variables: 'Emissions|CO2', 'Primary Energy|Coal', 
    #query for climate scenario data
    
    df = pyam.read_iiasa(
        name = 'engage_internal',
        creds = iiasa_creds,
        scenario =[
            "T34_1000_ref",
            "T34_1000_govem",
            "T34_1000_feas_em",
            "T34_1000_bitb_em",
            "T34_1000_bitb_ref",
            "T34_1000_enab_em"
        ],
        variable=["Emissions|CO2", 
                  'Emissions|Kyoto Gases',
                  'Primary Energy|Coal',
                  'Primary Energy|Gas',
                  'Primary Energy',
                  'Capacity|Electricity|Solar',
                  'Capacity|Electricity|Wind',
                  'Carbon Sequestration|CCS',
                  'Carbon Sequestration|Land Use'
                  ],
        region=["North America; primarily the United States of America and Canada",
                "Eastern and Western Europe (i.e., the EU28)",
                "Countries of centrally-planned Asia; primarily China",
                "Countries of South Asia; primarily India", 
                "Countries of Sub-Saharan Africa",
                "Countries of Latin America and the Caribbean",
                "Countries of the Middle East; Iran, Iraq, Israel, Saudi Arabia, Qatar, etc."],
        years = ["2020", "2025", "2030", "2035", "2040"]
    )   
    #return data format of df
    return df


In [125]:
data_set = get_data().data

httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications/IXSE_ENGAGE_INTERNAL/config "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected to the IXSE_ENGAGE_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.

In [126]:
df = data_set


In [144]:
#Data wrangling
df = data_set
#get regional groupings
## OECD: "North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)"
#TODO Clarify that countries of Asia and Latin America are missing
## China:  Countries of centrally-planned Asia; primarily China
## Rest of the world: other countries
#get OECD*
oecd = df[df["region"].isin(["North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)"])]\
    .groupby(["model", "scenario", "variable", "year", "unit"])\
        .agg({"value": "sum"}).reset_index()
oecd['region'] = "OECD"
#get China
china = df[df['region'] == "Countries of centrally-planned Asia; primarily China"]
china.loc[:, "region"] = "China"
#get RoW
row = df[~df["region"].isin(["North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)", "Countries of centrally-planned Asia; primarily China"])]\
        .groupby(["model", "scenario", "variable", "year", "unit"])\
            .agg({"value": "sum"}).reset_index()
row["region"] = "RoW"
#Concat three regions
df = pd.concat([oecd, china, row]).reset_index()

#from long to wide
df = pd.pivot(data=df, index=['model','scenario', 'region', 'year'], columns = 'variable', values = 'value').reset_index()
#get diffeernt scenario narratives from scenario name
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_ref", case=True), "Cost Effective", np.nan) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_bitb_ref", case=True), "Tech", df['scenario_narrative']) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_govem", case=True), "Instit", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_bitb_em", case=True), "Tech+Inst", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_enab_em", case=True), "Inst+Enab", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_feas_em", case=True), "Tech+Inst+Enab", df['scenario_narrative'])

#caluclate percentages of energymix for coal and gas
df["Share_Coal"] = df["Primary Energy|Coal"] / df["Primary Energy"]
df["Share_Gas"] = df["Primary Energy|Gas"] / df["Primary Energy"]


#Sort dataframe
df.sort_values(by=["model", "scenario", "region", "year"], ascending=True, inplace=True)

In [183]:
df.head(3)
df[(df["model"] == "MESSAGEix-GLOBIOM_1.1") & (df["scenario_narrative"] == "Tech+Inst") & (df["region"] == "China")].duplicated()

df[['year', "Emissions|CO2", "region", "scenario_narrative"]].duplicated().sum()

0

In [182]:
#Figure 1: CO2 trajectories
fig1 = px.line(df.sort_values(by=["model", "year"], ascending=False),
               x = "year",
               y="Emissions|CO2",
               color = "region",
               #line_dash="model",
               facet_col="scenario_narrative",
               facet_col_wrap= 3,
               range_x=[2005, 2100],
               range_y = [-10000, 15000],
               hover_data = {'scenario':True, 'model':True},
               title = "CO2 emission trajectories")


#fig1.update_layout(showlegend=False)
fig1

In [122]:
#Figure 2: GHG trajectories
fig2 = px.line(df,
               x = "year",
               y="Emissions|Kyoto Gases",
               color = "region",
               line_dash="model",
               facet_col="scenario_narrative",
               facet_col_wrap= 3,
               range_x=[2005, 2100],
               range_y = [-10000, 20000],
               hover_data = {'scenario':True, 'model':True}, 
               title = "GHG emission trajectories")
fig2.update_layout(showlegend=False)
fig2

In [31]:
#calculate year that each scenario hit's net zero
netzero_df = df[df["Emissions|CO2"] <= 0].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
netzero_df.rename(columns={"year": "year_netzero"}, inplace=True)
netzero_df
#aggregated cost until 2100
#df["year"] = pd.to_numeric(df['year'])
#df[df["year"] <= 2100].groupby(["model", "scenario"]).agg({"AR5 climate diagnostics|Temperature|Exceedance Probability|1.5 degC|MAGICC6": ['sum']})

,model,scenario,carbon_budget,year_netzero
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2060
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2070
3,GEM-E3_V2023,T34_1000_feas_em,2C,2090
4,IMAGE 3.2,T34_1000_feas_em,2C,2070
5,IMAGE 3.2,T34_1000_feas_pr,2C,2070
6,IMAGE 3.2,T34_1000_feas_ref,2C,2070
7,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,2080
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,2070
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_ref,2C,2100


In [10]:
fig2 = px.scatter(
    data_frame=netzero_df,
    x = "year",
    y="carbon_budget"
)
fig2

In [32]:
#get amounts of unabated coal
#both Coal data are in EJ/yr
unabcoal_df = df[["model", "scenario", "carbon_budget", "region","year", "Primary Energy|Coal", "Primary Energy|Coal|w/ CCS"]] 
unabcoal_df["Unabated Coal"] = unabcoal_df["Primary Energy|Coal"] - unabcoal_df["Primary Energy|Coal|w/ CCS"]
#pick threshold for when coal is supposed to the phased out // now it's 8EJ/y
unabcoal_df = unabcoal_df[unabcoal_df["Unabated Coal"] <= 8].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
unabcoal_df.rename(columns={"year": "year_netcoal"}, inplace=True)
#combine with netzero df

unabcoal_df

C:\Users\schei\AppData\Local\Temp\ipykernel_10492\1471908738.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,model,scenario,carbon_budget,year_netcoal
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2055
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2050
3,GEM-E3_V2023,T34_1000_feas_em,2C,2065
4,GEM-E3_V2023,T34_1000_feas_ref,2C,2080
5,IMAGE 3.2,T34_1000_feas_em,2C,2060
6,IMAGE 3.2,T34_1000_feas_pr,2C,2060
7,IMAGE 3.2,T34_1000_feas_ref,2C,2060
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,2060
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,2060


In [88]:
#combine dataframes
to_plot = pd.merge(left=netzero_df, right=unabcoal_df, on=["model", "scenario", "carbon_budget"])

fig3 = px.scatter(
    data_frame=to_plot,
    x = "year_netzero",
    y="year_netcoal",
    color= "carbon_budget"
)
fig3

In [89]:
#calculate change in transportation energy from 2020 to 2030
tranred_df = df[["model", "scenario", "carbon_budget", "region","year", "Final Energy|Transportation"]]
#filter for year 2020 & 2030
tranred_df = tranred_df[(tranred_df["year"] == 2020) | (tranred_df["year"] == 2030) ].reset_index().drop("index", axis=1)
#from long to wide
tranred_df = pd.pivot(data=tranred_df, index=["model", "scenario", "carbon_budget", "region"], 
             columns="year", values= "Final Energy|Transportation").reset_index()
#Finale Energy Transportation = (FET)
tranred_df["redu_FET"] = (tranred_df[2030] - tranred_df[2020]) / tranred_df[2020] 

# #both Coal data are in EJ/yr
# unabcoal_df = df[["model", "scenario", "carbon_budget", "region","year", "Primary Energy|Coal", "Primary Energy|Coal|w/ CCS"]] 
# unabcoal_df["Unabated Coal"] = unabcoal_df["Primary Energy|Coal"] - unabcoal_df["Primary Energy|Coal|w/ CCS"]
# #pick threshold for when coal is supposed to the phased out // now it's 8EJ/y
# unabcoal_df = unabcoal_df[unabcoal_df["Unabated Coal"] <= 8].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
# unabcoal_df.rename(columns={"year": "year_netcoal"}, inplace=True)

px.line(df,
        x = "year",
        y="Final Energy|Transportation",
        color = "carbon_budget",
        line_dash="model",
        facet_col="scenario_narrative",
        hover_data = {'scenario':True, 'model':True})

tranred_df

year,model,scenario,carbon_budget,region,2020,2030,redu_FET
0,AIM/CGE V2.2,T34_1000_feas_em,2C,World,123.601600,104.300200,-0.156158
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,World,123.601600,104.296500,-0.156188
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,World,123.601600,111.827600,-0.095258
3,GEM-E3_V2023,T34_1000_feas_em,2C,World,106.172003,107.829240,0.015609
4,GEM-E3_V2023,T34_1000_feas_ref,2C,World,106.172003,107.223507,0.009904
5,IMAGE 3.2,T34_1000_feas_em,2C,World,92.399406,78.249680,-0.153137
6,IMAGE 3.2,T34_1000_feas_pr,2C,World,92.399406,77.118727,-0.165376
7,IMAGE 3.2,T34_1000_feas_ref,2C,World,92.399406,85.292977,-0.076910
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,World,113.118992,140.618740,0.243105
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,World,113.118992,141.497312,0.250871


In [91]:
#merge reduction data to rest of df
to_plot = pd.merge(left=netzero_df, right=unabcoal_df, on=["model", "scenario", "carbon_budget"])
to_plot = pd.merge(left=to_plot, right=tranred_df.loc[:, ~tranred_df.columns.isin([2020, 2030])],
                   on=["model", "scenario", "carbon_budget"])
to_plot.head(3)

,model,scenario,carbon_budget,year_netzero,year_netcoal,region,redu_FET
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2060,2055,World,-0.156158
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055,2055,World,-0.156188
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2070,2050,World,-0.095258


In [86]:
fig4 = px.scatter(
    data_frame=to_plot,
    y = "redu_FET",
    x="year_netcoal",
    color= "carbon_budget"
)
fig4

In [102]:
#Statement
#Until when is it feasible to phase out coal
year_coal_phaseout = 2050
#What change in energy consumption of transport sector is realistic until 2030?
change_energy_transport = 0

#filter dataframe
filter_df = to_plot.loc[(to_plot["year_netcoal"] >= year_coal_phaseout) & (to_plot["redu_FET"] >= change_energy_transport)]
earliest_net_zero_year = filter_df["year_netzero"].min()
PA_aligned = (filter_df["carbon_budget"].str.contains("1.5C").sum() > 0)

print("OUTPUT:")
print("Earliest possible net-zero year:", earliest_net_zero_year)
print("Is PA still possible?", PA_aligned)

OUTPUT:
Earliest possible net-zero year: 2050
Is PA still possible? True
